In [21]:
#pip install chromadb
#!pip install pandas
#!pip install nltk


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     |████████████████████████████████| 1.5 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 44.0 MB/s eta 0:00:01
     |████████████████████████████████| 773 kB 58.1 MB/s eta 0:00:01


# Import Libraries

In [10]:
import pandas as pd
from tqdm import tqdm

In [11]:
#create a persistent chroma db
import chromadb
import pandas as pd
#create a persistentClient
client=chromadb.PersistentClient(path="ChromaDB")

In [12]:
# function to chunk


def new_chunker(chunk_size, id, content,collection,new_id,new_content):
    
    tokens = [token for token in content.split()]
    # Get length of the tokens
    n = len(tokens)

    # Initialize index and count
    index, count = 0, 0
    # Print(id,content,tokens)
    while True:
        if index == 0:
            si, ei = index, 256
            if ei > n:
                new_id.append(f"{id}-{count}")
                new_content.append(" ".join(tokens[si:]))
                break
            else:
                new_id.append(f"{id}-{count}")
                new_content.append(" ".join(tokens[si:ei]))
                index = ei
        else:
            si, ei = index - chunk_size, index + 256 - chunk_size  # chunking buffer
            if ei > n:
                new_id.append(f"{id}-{count}")
                new_content.append(" ".join(tokens[si:]))
                break
            else:
                new_id.append(f"{id}-{count}")
                new_content.append(" ".join(tokens[si:ei]))
                index = ei
        count += 1

In [33]:
from tqdm.notebook import tqdm
import pandas as pd
from math import ceil

def Chunker_Uploader(chunk_size, collection_name, file_path):

    # Data Ingestion
    df = pd.read_csv(file_path, index_col=0)

    
    # Create or get a collection
    collection = client.get_or_create_collection(name=collection_name, metadata={"hnsw:space": "cosine"})

    # Create two lists
    new_id = []
    new_content = []
    print('Chunking Started!')
    # Chunk and upload
    for i in df.index:
        if type(df.loc[i,'pre-processed_content'])==str:
            new_chunker(chunk_size, i, df.loc[i, 'pre-processed_content'], collection, new_id, new_content)
        
    print('Chunking Ended',i)
    pass_size = 10000
    passes=ceil(len(new_id)/10000)
    print('No of passes',passes)
    for i in range(passes):
        print('Current Pass no:',i+1)
        si = 0+(i*pass_size)
        ei = min(si + pass_size, len(new_id) ) # Ensure ei does not exceed the list length
        
        curr_id = new_id[si:ei]
        curr_content = new_content[si:ei]
        # Initialize tqdm
        progress_bar = tqdm(total=len(curr_id), desc="Adding vectors", unit="doc")

        # Add chunks to the collection
        for new_doc_id, new_doc_content in zip(curr_id, curr_content):
            # Add document to the collection
            collection.add(ids=new_doc_id, documents=new_doc_content)

            # Update progress bar
            progress_bar.update(1)

        progress_bar.close()

        print('Vectors are added!!!')


In [34]:
Chunker_Uploader(50,'transcripts',"70000-80000eng_subtitles.csv")

Chunking Started!
Chunking Ended 79999
No of passes 13
Current Pass no: 3


Adding vectors:   0%|          | 0/10000 [00:00<?, ?doc/s]

Insert of existing embedding ID: 71626-8
Add of existing embedding ID: 71626-8
Insert of existing embedding ID: 71626-9
Add of existing embedding ID: 71626-9
Insert of existing embedding ID: 71626-10
Add of existing embedding ID: 71626-10
Insert of existing embedding ID: 71627-0
Add of existing embedding ID: 71627-0
Insert of existing embedding ID: 71627-1
Add of existing embedding ID: 71627-1
Insert of existing embedding ID: 71627-2
Add of existing embedding ID: 71627-2
Insert of existing embedding ID: 71627-3
Add of existing embedding ID: 71627-3
Insert of existing embedding ID: 71627-4
Add of existing embedding ID: 71627-4
Insert of existing embedding ID: 71627-5
Add of existing embedding ID: 71627-5
Insert of existing embedding ID: 71627-6
Add of existing embedding ID: 71627-6
Insert of existing embedding ID: 71627-7
Add of existing embedding ID: 71627-7
Insert of existing embedding ID: 71627-8
Add of existing embedding ID: 71627-8
Insert of existing embedding ID: 71627-9
Add of ex

KeyboardInterrupt: 

In [ ]:
#Chunker_Uploader(0,'demo',"10000-20000eng_subtitles.csv")